<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Улучшим-качество-моделей-учтя-дисбаланс." data-toc-modified-id="Улучшим-качество-моделей-учтя-дисбаланс.-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Улучшим качество моделей учтя дисбаланс.</a></span></li><li><span><a href="#Попробуем-увеличить-выборку-(upsampling)." data-toc-modified-id="Попробуем-увеличить-выборку-(upsampling).-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Попробуем увеличить выборку (upsampling).</a></span></li><li><span><a href="#Поробуем-чуть-чуть-улучшить-модель-гиперпараметрами." data-toc-modified-id="Поробуем-чуть-чуть-улучшить-модель-гиперпараметрами.-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Поробуем чуть-чуть улучшить модель гиперпараметрами.</a></span></li><li><span><a href="#Попробуем-уменьшить-выборку-(downsampling)." data-toc-modified-id="Попробуем-уменьшить-выборку-(downsampling).-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Попробуем уменьшить выборку (downsampling).</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

# Отток клиентов

Из банка стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from tqdm import tqdm

pd.options.mode.chained_assignment = None

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


В столбце `Tenure` есть пропуски. Заполним пропуски медианным значением.

In [3]:
data.loc[data['Tenure'].isna(), ['Tenure']] = data['Tenure'].median()

Столбцы `RowNumber`, `Surname`, `CustomerId` содержат уникальные значения. Фамилия, индекс, ID. Для прогноза они роли не играют, можем их удалить. Идентифицировать конкретного пользователя можно по индексу датафрейма, который совпадает с индексом в столбце `RowNumber`.

In [4]:
data_ohe = data.drop(['RowNumber', 'Surname', 'CustomerId'], axis=1)
data_ohe = pd.get_dummies(data_ohe, drop_first=True)

In [5]:
features = data_ohe.drop(['Exited'], axis=1)
target = data_ohe['Exited']

In [6]:
features_train, features_test, target_train, target_test = \
train_test_split(features, target, test_size=0.20, random_state=54321)
features_train, features_valid, target_train, target_valid = \
train_test_split(features_train, target_train, test_size=0.25, random_state=54321)

In [7]:
features_train.shape, features_valid.shape, features_test.shape, target_train.shape, target_valid.shape, target_test.shape

((6000, 11), (2000, 11), (2000, 11), (6000,), (2000,), (2000,))

Маштабируем числовые признаки в столбцах `CreditScore`, `Age`, `Tenure`, `Balance`, `EstimatedSalary`. Остальные столбцы в масштабировании не нуждаются. Данные в небольшом диапазоне.

In [8]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

**Вывод**  
После загрузки данных, я:
- удалил столбцы содержащие уникальную персональную информацию не влияющую на анализ. 
- пропущенные данные заполнил медианным значением. 
- разделил на обущающую, валидационную выборки и тестовую, в соотношении 3:1:1. 
- масштабировал столбцы содержащие большие числа.  

**Данные подготовлены.**

## Исследование задачи

Обучим модель Логистическая регрессия и посчитаем метрику `accuracy`.

In [9]:
model_LR = LogisticRegression(solver='liblinear', random_state=54321)
model_LR.fit(features_train, target_train)
prediction_valid_LR = model_LR.predict(features_valid)
accuracy_score(target_valid, prediction_valid_LR)

0.799

Проверим сколько и каких значений содержит целевой признак, что бы понять насколько сбалансирован класс. Посчитаем метрику `accuracy` для модели содержащий наиболее частое значение. 

In [10]:
target_valid.value_counts()

0    1558
1     442
Name: Exited, dtype: int64

Класс не сбалансирован. Значений `0` в 3,5 раза больше значений `1`. Модель выдающая только `0` покажет `accuracy` равным:

In [11]:
1558/(1558+442)

0.779

**Вывод**  
Обучили модель Логистическая регрессия. Модель показала значение accuracy раное 79,9%. С помощью константной модели проверили адекватность. Accuracy константной модели равна 77,9%. Наша модель неадекватна. Использовать эту модель бесполезно. Использовать метрику accuracy нельзя. 
Такой результат получили, потому что целевой признак не сбалансирован.


In [12]:
def print_f1_auc_roc(name, model):
    f1 = f1_score(target_valid, model.predict(features_valid))
    roc_a = roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1])
    if not name :
        return f1, roc_a
    print(f'Метрика F1 для {name} равна {f1}')
    print(f'Метрика auc-roc для {name} равна {roc_a}')    

In [13]:
print_f1_auc_roc('логистической регрессии', model_LR)

Метрика F1 для логистической регрессии равна 0.31864406779661014
Метрика auc-roc для логистической регрессии равна 0.7576310271318955


Попробуем подобрать гиперпараметры у модели логистическая регресия.

In [14]:
print_f1_auc_roc('логистической регрессии', LogisticRegression(solver='liblinear', max_iter=1000, random_state=54321).\
                 fit(features_train, target_train))

Метрика F1 для логистической регрессии равна 0.31864406779661014
Метрика auc-roc для логистической регрессии равна 0.7576310271318955


In [15]:
print_f1_auc_roc('логистической регрессии', LogisticRegression(solver='lbfgs', random_state=54321).\
                 fit(features_train, target_train))

Метрика F1 для логистической регрессии равна 0.31578947368421056
Метрика auc-roc для логистической регрессии равна 0.7577936674818047


Изменение гиперпараметра max_iter на результат не повлияло, изменение гиперпараметра solver с liblinear на lbfgs слегка улучшило результат

Обучим модель дерево решений без учета дисбаланса.

In [16]:
print_f1_auc_roc('дерева решений', DecisionTreeClassifier(random_state=54321).fit(features_train, target_train))

Метрика F1 для дерева решений равна 0.5368536853685368
Метрика auc-roc для дерева решений равна 0.7044519891495653


Модель дерево решений показывает лучшие результаты, попробуем подобрать гиперпараметры.

In [17]:
f1_best, ar_best = 0, 0
res = []
for depth in range(1, 41):
    f1_dt, ar_dt = print_f1_auc_roc('',  DecisionTreeClassifier(max_depth=depth, random_state=54321).\
                                    fit(features_train, target_train))
    if (f1_dt > f1_best):
        f1_best = f1_dt
        res.append(('f1', depth, f1_best, ar_best))
    if (ar_dt > ar_best):
        ar_best = ar_dt
        res.append(('ar', depth, f1_best, ar_best))
res

[('ar', 1, 0, 0.6804872240196562),
 ('f1', 2, 0.5049928673323822, 0.6804872240196562),
 ('ar', 2, 0.5049928673323822, 0.7405806550920951),
 ('f1', 3, 0.5424200278164116, 0.7405806550920951),
 ('ar', 3, 0.5424200278164116, 0.7978547447417793),
 ('ar', 4, 0.5424200278164116, 0.8154968662689722),
 ('f1', 5, 0.5714285714285714, 0.8154968662689722),
 ('ar', 5, 0.5714285714285714, 0.8354587910013417),
 ('f1', 6, 0.5885558583106267, 0.8354587910013417),
 ('ar', 6, 0.5885558583106267, 0.8425242072735089)]

Лучшие результаты показывает модель с гиперпараметром max_depth = 6.

Лучшие результаты из рассмотренных моделей без учета дисбаланса показала модель решаюшее дерево с подобранными гиперпараметрами.  
f1 = 0.5885558583106267, auc-roc = 0.8425242072735089

## Борьба с дисбалансом

### Улучшим качество моделей учтя дисбаланс. 

Используем взвешивание классов. Для оценки результата будем использовать метрику F1.

Рассмотрим модель логистическая регрессия.

In [18]:
model_LR = LogisticRegression(solver='liblinear', random_state=54321, class_weight='balanced')
model_LR.fit(features_train, target_train)
prediction_valid = model_LR.predict(features_valid)
f1_score(target_valid, prediction_valid)

0.5192629815745393

Рассмотрим модель дерево решений

In [19]:
model_DT = DecisionTreeClassifier(random_state=54321, class_weight='balanced')
model_DT.fit(features_train, target_train)
prediction_valid = model_DT.predict(features_valid)
f1_score(target_valid, prediction_valid)

0.5017667844522968

Рассмотрим модель случайный лес

In [20]:
model_RF = RandomForestClassifier(random_state=54321, class_weight='balanced')
model_RF.fit(features_train, target_train)
prediction_valid = model_RF.predict(features_valid)
f1_score(target_valid, prediction_valid)

0.544395924308588

In [21]:
print_f1_auc_roc('логистической регрессии', LogisticRegression(solver='liblinear', random_state=54321, class_weight='balanced').\
                 fit(features_train, target_train))

Метрика F1 для логистической регрессии равна 0.5192629815745393
Метрика auc-roc для логистической регрессии равна 0.7647683246301386


Подберем гиперпараметы

In [22]:
print_f1_auc_roc('логистической регрессии', LogisticRegression(solver='lbfgs', random_state=54321, class_weight='balanced').\
                 fit(features_train, target_train))

Метрика F1 для логистической регрессии равна 0.51802179379715
Метрика auc-roc для логистической регрессии равна 0.7646942651850905


Выбор лучших гиперпараметров, полученных без учета дисбаланса, ухудшил результаты модели, когда дисбаланс учли.  
Вывод: при учете дисбаланса, набор лучших гиперпараметров может поменяться.

Обучим дерево решений на сбалансированной выборке

In [23]:
print_f1_auc_roc('дерева решений', DecisionTreeClassifier(random_state=54321, class_weight='balanced').\
                 fit(features_train, target_train))

Метрика F1 для дерева решений равна 0.5017667844522968
Метрика auc-roc для дерева решений равна 0.6786909194407496


Подберем гиперпараметы для дерева решений

In [24]:
f1_best, ar_best = 0, 0
res = []
for depth in range(1, 41):
    f1_dt, ar_dt = print_f1_auc_roc('',  DecisionTreeClassifier(max_depth=depth, class_weight='balanced', random_state=54321).\
                                    fit(features_train, target_train))
    if (f1_dt > f1_best):
        f1_best = f1_dt
        res.append(('f1', depth, f1_best, ar_best))
    if (ar_dt > ar_best):
        ar_best = ar_dt
        res.append(('ar', depth, f1_best, ar_best))
res

[('f1', 1, 0.5058087578194816, 0),
 ('ar', 1, 0.5058087578194816, 0.6936915874279009),
 ('f1', 2, 0.5403996524761077, 0.6936915874279009),
 ('ar', 2, 0.5403996524761077, 0.7492942570530731),
 ('f1', 3, 0.5497287522603979, 0.7492942570530731),
 ('ar', 3, 0.5497287522603979, 0.7952082086908032),
 ('f1', 4, 0.5520504731861198, 0.7952082086908032),
 ('ar', 4, 0.5520504731861198, 0.8148601002561586),
 ('ar', 5, 0.5520504731861198, 0.8173069662347016),
 ('f1', 6, 0.5822102425876011, 0.8173069662347016)]

Лучший результат показывает модель с гиперпараметром max_depth = 6

In [25]:
print_f1_auc_roc('дерева решений', DecisionTreeClassifier(random_state=54321, max_depth = 6, class_weight='balanced').\
                 fit(features_train, target_train))

Метрика F1 для дерева решений равна 0.5822102425876011
Метрика auc-roc для дерева решений равна 0.8107294710122619


Обучим случайный лес на сбалансированной выборке

In [26]:
print_f1_auc_roc('случайного леса', RandomForestClassifier(random_state=54321, class_weight='balanced').\
                 fit(features_train, target_train))

Метрика F1 для случайного леса равна 0.544395924308588
Метрика auc-roc для случайного леса равна 0.8483923291840683


С подобранными гиперпараметрами случайный лес показывает следующие результаты

In [40]:
f1, f1_best, depth_best, model_RF_best = 0, 0, 1, 0
for depth in tqdm(range(1, 40)):
    model_RF = RandomForestClassifier(random_state=54321, max_depth=depth, class_weight='balanced')
    model_RF.fit(features_train, target_train)
    prediction_valid = model_RF.predict(features_valid)
    f1 = f1_score(target_valid, prediction_valid)
    if f1 > f1_best:
        f1_best = f1
        depth_best = depth
print('---------')        
for est in tqdm(range(20, 101)):
    model_RF = RandomForestClassifier(n_estimators=est, random_state=54321, max_depth=depth_best, class_weight='balanced')
    model_RF.fit(features_train, target_train)
    prediction_valid = model_RF.predict(features_valid)
    f1 = f1_score(target_valid, prediction_valid)
    if f1 > f1_best:
        f1_best = f1
        model_RF_best = model_RF
        est_best = est
print(f'F1 = {f1_best}, Количество деревьев = {est_best}, глубина = {depth_best}')

100%|██████████| 39/39 [00:35<00:00,  1.10it/s]


---------


100%|██████████| 81/81 [00:38<00:00,  2.08it/s]

F1 = 0.6389452332657201, Количество деревьев = 78, глубина = 9


In [27]:
print_f1_auc_roc('случайного леса', RandomForestClassifier(n_estimators=78, random_state=54321, max_depth=9, class_weight='balanced').\
                 fit(features_train, target_train))

Метрика F1 для случайного леса равна 0.6224156692056584
Метрика auc-roc для случайного леса равна 0.8603964939387427


Лучшее значение дает модель случайный лес F1 = 0.6224156692056584, auc-roc = 0.8603064608879001.

Лучшее значение дает модель случайный лес - 0.544

### Попробуем увеличить выборку (upsampling).

Разобъем обучающую выборку по целевому признаку на 0 и 1. Количество 1 увеличим пропорционально 0. Перемешаем.

In [28]:
features_train_0 = features_train[target_train == 0]
features_train_1 = features_train[target_train == 1]
target_train_0 = target_train[target_train == 0]
target_train_1 = target_train[target_train == 1]
repeat = round(len(target_train_0)/len(target_train_1))

In [29]:
features_train_up = pd.concat([features_train_0] + [features_train_1] * repeat)
target_train_up = pd.concat([target_train_0] + [target_train_1] * repeat)
features_train_up, target_train_up = shuffle(features_train_up, target_train_up, random_state=54321)

Обучим модели на увеличенной выборке

In [30]:
model_LR.fit(features_train_up, target_train_up)
prediction_valid = model_LR.predict(features_valid)
print('Логистическая регрессия. F1 =', f1_score(target_valid, prediction_valid))

Логистическая регрессия. F1 = 0.5192629815745393


In [31]:
model_DT.fit(features_train_up, target_train_up)
prediction_valid = model_DT.predict(features_valid)
print('Решающее дерево. F1 =', f1_score(target_valid, prediction_valid))

Решающее дерево. F1 = 0.5125284738041002


In [32]:
model_RF.fit(features_train_up, target_train_up)
prediction_valid = model_RF.predict(features_valid)
print('Случайный лес. F1 =', f1_score(target_valid, prediction_valid))

Случайный лес. F1 = 0.6039119804400978


In [33]:
print_f1_auc_roc('логистической регрессии', LogisticRegression(solver='liblinear', random_state=54321).\
                 fit(features_train_up, target_train_up))

Метрика F1 для логистической регрессии равна 0.5179615705931494
Метрика auc-roc для логистической регрессии равна 0.7647189516667732


In [34]:
print_f1_auc_roc('дерева решений', DecisionTreeClassifier(random_state=54321).\
                 fit(features_train_up, target_train_up))

Метрика F1 для дерева решений равна 0.49706916764361075
Метрика auc-roc для дерева решений равна 0.6759550764119215


In [35]:
print_f1_auc_roc('случайного леса', RandomForestClassifier(random_state=54321).\
                 fit(features_train_up, target_train_up))

Метрика F1 для случайного леса равна 0.5990099009900991
Метрика auc-roc для случайного леса равна 0.850618468973449


Подберем гиперпараметры

In [37]:
f1_best, ar_best = 0, 0
res = []
for depth in range(1, 41):
    f1_dt, ar_dt = print_f1_auc_roc('',  DecisionTreeClassifier(max_depth=depth, random_state=54321).\
                                    fit(features_train_up, target_train_up))
    if (f1_dt > f1_best):
        f1_best = f1_dt
        res.append(('f1', depth, f1_best, ar_best))
    if (ar_dt > ar_best):
        ar_best = ar_dt
        res.append(('ar', depth, f1_best, ar_best))
res

[('f1', 1, 0.5058087578194816, 0),
 ('ar', 1, 0.5058087578194816, 0.6936915874279009),
 ('f1', 2, 0.5403996524761077, 0.6936915874279009),
 ('ar', 2, 0.5403996524761077, 0.7492942570530731),
 ('f1', 3, 0.5497287522603979, 0.7492942570530731),
 ('ar', 3, 0.5497287522603979, 0.7952082086908032),
 ('f1', 4, 0.5520504731861198, 0.7952082086908032),
 ('ar', 4, 0.5520504731861198, 0.8148601002561586),
 ('ar', 5, 0.5520504731861198, 0.8173069662347016),
 ('f1', 6, 0.5822102425876011, 0.8173069662347016)]

In [38]:
print_f1_auc_roc('дерева решений', DecisionTreeClassifier(max_depth=6, random_state=54321).\
                 fit(features_train_up, target_train_up))

Метрика F1 для дерева решений равна 0.5822102425876011
Метрика auc-roc для дерева решений равна 0.8107294710122619


In [42]:
f1, f1_best, depth_best, model_RF_best = 0, 0, 1, 0
for depth in tqdm(range(1, 40)):
    model_RF = RandomForestClassifier(random_state=54321, max_depth=depth)
    model_RF.fit(features_train_up, target_train_up)
    prediction_valid = model_RF.predict(features_valid)
    f1 = f1_score(target_valid, prediction_valid)
    if f1 > f1_best:
        f1_best = f1
        depth_best = depth
print('---------')        
for est in tqdm(range(20, 101)):
    model_RF = RandomForestClassifier(n_estimators=est, random_state=54321, max_depth=depth_best)
    model_RF.fit(features_train_up, target_train_up)
    prediction_valid = model_RF.predict(features_valid)
    f1 = f1_score(target_valid, prediction_valid)
    if f1 > f1_best:
        f1_best = f1
        model_RF_best = model_RF
        est_best = est
print(f'F1 = {f1_best}, Количество деревьев = {est_best}, глубина = {depth_best}')

100%|██████████| 39/39 [00:33<00:00,  1.17it/s]


---------


100%|██████████| 81/81 [00:34<00:00,  2.32it/s]

F1 = 0.6316831683168317, Количество деревьев = 78, глубина = 8


Лучшие гиперпараметры для модели случайный лес обученной на увеличенной выборке. Деревьев - 78, глубина 8.

In [46]:
print_f1_auc_roc('случайного леса', RandomForestClassifier(n_estimators=78, max_depth=8, random_state=54321).\
                 fit(features_train_up, target_train_up))

Метрика F1 для случайного леса равна 0.6214355948869223
Метрика auc-roc для случайного леса равна 0.8619531944307297


Лучший результат показывает модель `случайный лес`.  

### Поробуем чуть-чуть улучшить модель гиперпараметрами.

In [53]:
f1, f1_best, depth_best, model_RF_best = 0, 0, 1, 0
for depth in tqdm(range(1, 40)):
    model_RF = RandomForestClassifier(random_state=54321, max_depth=depth, class_weight='balanced')
    model_RF.fit(features_train_up, target_train_up)
    prediction_valid = model_RF.predict(features_valid)
    f1 = f1_score(target_valid, prediction_valid)
    if f1 > f1_best:
        f1_best = f1
        depth_best = depth
print('---------')        
for est in tqdm(range(20, 101)):
    model_RF = RandomForestClassifier(n_estimators=est, random_state=54321, max_depth=depth_best, class_weight='balanced')
    model_RF.fit(features_train_up, target_train_up)
    prediction_valid = model_RF.predict(features_valid)
    f1 = f1_score(target_valid, prediction_valid)
    if f1 > f1_best:
        f1_best = f1
        model_RF_best = model_RF
        est_best = est
print(f'F1 = {f1_best}, Количество деревьев = {est_best}, глубина = {depth_best}')

100%|██████████| 39/39 [00:33<00:00,  1.15it/s]


---------


100%|██████████| 81/81 [00:36<00:00,  2.19it/s]

F1 = 0.6389452332657201, Количество деревьев = 78, глубина = 9


Лучшая модель `RandomForestClassifier` с количеством деревьев 78 и глубиной 9.

### Попробуем уменьшить выборку (downsampling).

In [43]:
division = len(target_train_1)/len(target_train_0)

Уменьшим выборку, перемешаем.

In [47]:
features_train_down = pd.concat([features_train_0.sample(frac=division, random_state=54321)] + [features_train_1])
target_train_down = pd.concat([target_train_0.sample(frac=division, random_state=54321)] + [target_train_1])
features_train_down, target_train_down = shuffle(features_train_down, target_train_down, random_state=54321)

Обучим лучшую модель на уменьшеной выборке.

In [48]:
model_RF_best_down = RandomForestClassifier(n_estimators=78, random_state=54321, max_depth=9)
model_RF_best_down.fit(features_train_down, target_train_down)
prediction_valid = model_RF_best_down.predict(features_valid)
print('Случайный лес. F1 =', f1_score(target_valid, prediction_valid))

Случайный лес. F1 = 0.6039426523297491


In [51]:
print_f1_auc_roc('случайного леса', RandomForestClassifier(n_estimators=78, max_depth=8, random_state=54321).\
                 fit(features_train_down, target_train_down))

Метрика F1 для случайного леса равна 0.6116152450090744
Метрика auc-roc для случайного леса равна 0.8605765600404278


Модель показывает снижение значения метрики. Уменьшение выборки в данном случае на пользу не пошло.

**Вывод**  
Выбранная модель - Случайный лес из 78 деревьев с глубиной 9. Значение метрики F1 = 0.6389452332657201.  

## Тестирование модели

Протестируем нашу лучшую модель и вычислим метрику AUC-ROC.

In [54]:
prediction_test = model_RF_best.predict(features_test)
print('Случайный лес. F1 =', f1_score(target_test, prediction_test))

Случайный лес. F1 = 0.6461187214611872


Вычислим AUC-ROC.

In [55]:
pos_ver = model_RF_best.predict_proba(features_valid)[:,1]

In [56]:
roc_auc_score(target_valid, pos_ver)

0.8590619717818994

Значение метрики AUC-ROC равно 0.8590619717818994

**Вывод**  
Лучшие результаты показала модель случайный лес с подобранными гиперпараметрами.  
На тестовой выборке F1 = 0.6461187214611872, AUC-ROC = 0.8590619717818994.  
Удалось достичь F1-меры равной ~ 0.65  
В целовом признаке был дисбаланс, без использования специальных техник, достичь результата не удалось бы.

**Вывод**  
В этом проекте нам нужно было спрогнозировать, уйдёт клиент из банка в ближайшее время или нет.  
Мы подготовили данные. Заполнили пропуски. Исключили ненужные столбцы. Применили прямое кодирование и масштабирование данных.  
Обучили модель без учета дисбаланса. Проверили баланс классов.  
Сбалансировали 3 способами целевой класс и обучили 3 модели.  
Выбрали лучшую и протестировали ее.  
Достигли F1-меры 0,646, исследовали метрику AUC-ROC.
Лучшая модель Случайный лес из 78 деревьев с глубиной 9.